# Pyomo import

In [1]:
from pyomo.environ import *

In [2]:
import random

# Knapsack Problem

## Formulação matemática

$$ maximizar \sum_{i \in I}{x_i . v_i} $$

$$ \sum_{i \in I}{x_i . c_i} \leq C $$


In [3]:
knapsack = ConcreteModel()

In [4]:
knapsack.Items = Set(initialize=[i for i in range(10)], doc='itens disponíveis')

In [5]:
def RandomRule(model, i):
    return random.random()

knapsack.Valores = Param(knapsack.Items, initialize=RandomRule, within=PositiveReals, mutable=False, doc='Valores dos itens')
knapsack.Custos = Param(knapsack.Items, initialize=RandomRule, within=PositiveReals, mutable=False, doc='Custo dos itens')
knapsack.Capacidade = Param(initialize=-1, mutable=True, doc='Capacidade total')

In [6]:
knapsack.Valores.display()

Valores : Valores dos itens
    Size=10, Index=Items, Domain=PositiveReals, Default=None, Mutable=False
    Key : Value
      0 :   0.9915927988433182
      1 :   0.8706401512232174
      2 :  0.06731238402405126
      3 :   0.6506889484063654
      4 :  0.05507460443823664
      5 :   0.7605023641766407
      6 :   0.9281082602628737
      7 :  0.39038163722887376
      8 :  0.36107620802967355
      9 : 0.017989029699538528


In [7]:
knapsack.x = Var(knapsack.Items, within=Binary, doc='variável de decisão binária indicando a escolha ou não de cada item')

In [8]:
def ObjRule(model):
    return sum([model.x[i]*model.Valores[i] for i in model.Items])

knapsack.objetivo = Objective(rule=ObjRule, sense=maximize, doc='função objetivo: maximizar o valor dos itens selecionados')

In [9]:
def RestCap(model):
    return sum([model.x[i]*model.Custos[i] for i in model.Items]) <= model.Capacidade

knapsack.total_cap = Constraint(rule=RestCap, doc='restrição para capacidade total')

In [10]:
def solve_model(modelo, capacidade):
    modelo.Capacidade.set_value(capacidade)
    
    opt = SolverFactory('glpk')
    
    results = opt.solve(modelo)
    
    results.write()
    modelo.x.display()
    
    return modelo

In [11]:
solve_model(knapsack, 2)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3.84940082566419
  Upper bound: 3.84940082566419
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 11
  Number of nonzeros: 11
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.023633480072021484
# ----------------------------------------------------------
#   Solution Information
# ------------